In [3]:
'''
Support Vector Machine (SVM) with an Independent Implementation on Optimization Algorithm 

In this example, we will download the MNIST handwritten digits and create a soft-SVM to 
predict the digit category (0-9)
'''
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.framework import ops
ops.reset_default_graph() 
import soft_SVM

# Load data
data_dir = 'temp'
mnist = input_data.read_data_sets(data_dir, one_hot=False)

# Convert images into 28x28 (they are downloaded as 1x784)
train_xdata = np.array([np.reshape(x, (28, 28)) for x in mnist.train.images])
test_xdata = np.array([np.reshape(x, (28, 28)) for x in mnist.test.images]) 

train_data = np.reshape(train_xdata, (train_xdata.shape[0],train_xdata.shape[1]*train_xdata.shape[2])) 
print("train_data.shape = ", train_data.shape) 
test_data = np.reshape(test_xdata, (test_xdata.shape[0],test_xdata.shape[1]*test_xdata.shape[2]))

# Convert labels into one-hot encoded vectors
train_labels = mnist.train.labels
test_labels = mnist.test.labels 

# This number indicates percentage of training data used to train the model. When p = 1, 100 percent
# of data will be used. The testing data is always applied in 100 percent. 
p = 0.02
    
rows = train_data.shape[0]
columns = train_data.shape[1] 
solution = np.zeros((10, columns+1), dtype = float) 
for k in range(9): 
    # 
    # preparing data for label k
    # 
    row_0 = 0; 
    for i in range(rows):
        if train_labels[i] == k: 
            row_0 = row_0+1
    row_0 = int(row_0*p)
    A_0 = np.zeros((row_0, columns), dtype = float) 
    n = 0
    for i in range(rows): 
        if train_labels[i] == k and n < row_0:
            for j in range(columns): 
                A_0[n][j] = train_data[i][j]
            n = n+1 
    #
    # preparing data for lebal greater than k
    # 
    row_1 = 0; 
    for i in range(rows):
        if train_labels[i] > k: 
            row_1 = row_1+1      
    row_1 = int(row_1*p)
    A_1 = np.zeros((row_1, columns), dtype = float) 
    n = 0
    for i in range(rows): 
        if train_labels[i] > k and n < row_1:
            for j in range(columns): 
                A_1[n][j] = train_data[i][j]
            n = n+1

    x = np.zeros((columns+1+row_0+row_1,), dtype = float)
    flag = np.zeros((1,), dtype = int) 
    # 
    # calling personal SVM implememntation in c++
    #
    value = soft_SVM.optimization(A_0,A_1,x,flag) 
    print("optimal value = ", value) 
    for j in range(columns+1): 
        solution[k][j] = x[j] 
    

rows = test_data.shape[0] 
total = 0 
wrong = 0 
right = 0 
temp = np.zeros((9,), dtype = float)
for i in range(rows):
    for k in range(9):
        temp[k] = 0 
        for j in range(columns):
            temp[k] = temp[k]+test_data[i][j]*solution[k][j] 
        temp[k] = temp[k]+solution[k][columns]
    nine_indicator = 1
    for k in range(9): 
        if temp[k] >= 0.0: 
            if test_labels[i] == k: 
                right = right+1 
            else: 
                wrong = wrong+1
            nine_indicator = 0 
            break 
    if nine_indicator: 
        if test_labels[i] == 9: 
            right = right+1; 
        else: 
            wrong = wrong+1
            
print("wrong = ", wrong) 
print("right = ", right) 
print("accuracy = ", float(right)/float(rows)) 
      

Extracting temp/train-images-idx3-ubyte.gz
Extracting temp/train-labels-idx1-ubyte.gz
Extracting temp/t10k-images-idx3-ubyte.gz
Extracting temp/t10k-labels-idx1-ubyte.gz
train_data.shape =  (55000, 784)
optimal value =  1.905971168180528
optimal value =  3.8427128694039943
optimal value =  4.401461971028225
optimal value =  3.7530997667248056
optimal value =  5.049703743675781
optimal value =  3.942874136025304
optimal value =  0.745757813731908
optimal value =  2.287323211168423
optimal value =  0.6620989283181331
wrong =  1699
right =  8301
accuracy =  0.8301
